# Calculating Runs of  Homozygosity Along the Autosomal Chromosomes

This task will be performed using the "allel.roh_mhmm()" function in scikit-alle python library.

In [ ]:
!pip install seaborn


In [ ]:
# Improting libraries
import allel
import zarr
import numcodecs
import pandas as pd
from hmmlearn import hmm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# providing path to the zarr files
allChrs_zarr_path = '/data/gunarathnai/Ans_GA/gatk_variants/YmEthSmlInd_variants/Zarr_files/YmEthInd_allChrs_fltpass.zarr'
chrx_zarr_path = '/data/gunarathnai/Ans_GA/gatk_variants/YmEthSmlInd_variants/Zarr_files/YmEthInd_Chrx_BAP.zarr'
chr2_zarr_path = '/data/gunarathnai/Ans_GA/gatk_variants/YmEthSmlInd_variants/Zarr_files/YmEthInd_Chr2_BAP.zarr'
chr3_zarr_path = '/data/gunarathnai/Ans_GA/gatk_variants/YmEthSmlInd_variants/Zarr_files/YmEthInd_Chr3_BAP.zarr'

In [ ]:
# Opening actual zarr files and reading in the variant data into the environment
chrx_callset = zarr.open_group('YmEthInd_Chrx_BAP.zarr', mode='r')
chrx_callset
chr2_callset = zarr.open_group('YmEthInd_Chr2_BAP.zarr', mode='r')
chr2_callset
chr3_callset = zarr.open_group('YmEthInd_Chr3_BAP.zarr', mode='r')
chr3_callset

In [ ]:
# Don't keep this snippt without commenting out
# chr3_callset.tree(expand=True)

In [ ]:
# Load genotype data (GT - shape: variants x samples x ploidy)
chrx_genotypes = allel.GenotypeArray(chrx_callset['calldata/GT'])
chr2_genotypes = allel.GenotypeArray(chr2_callset['calldata/GT'])
chr3_genotypes = allel.GenotypeArray(chr3_callset['calldata/GT'])

# Load variant positions
chrx_positions = chrx_callset['variants/POS'][:]
chr2_positions = chr2_callset['variants/POS'][:]
chr3_positions = chr3_callset['variants/POS'][:]

# Chromosome lengths are give as the contig sizes to calculate the fraction of RoH
# These values are taken from the NCBI reference sequence for An. stephensi - accession ID - GCF_013141755.1
# Depending on the organism this should change
chrx_size=22713616
chr2_size=93706023
chr3_size=88747589


### Extract Population Data
RoH will be calculated for each population, which means we need to group individuals by population and extract phased haplotype data. Assuming you have predefined population groupings:

In [ ]:
# Load sample names (optional, for population assignment)
samples = chrx_callset['samples'][:]
len(samples)

# Define populations (indices of individuals in each population)
India_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]  # Replace with indices of population 1
Ethiopia_indices = [20, 21, 22, 23, 24, 25, 26, 27, 37, 38, 39, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55]  # Replace with indices of population 2
Yemen_indices = [28, 29, 30, 31, 32, 33, 34, 35, 36, 40, 41, 42]  # Replace with indices of population 3

# Subset genotypes by population
chrx_genotypes_India = chrx_genotypes.take(India_indices, axis=1)
chr2_genotypes_India = chr2_genotypes.take(India_indices, axis=1)
chr3_genotypes_India = chr3_genotypes.take(India_indices, axis=1)

chrx_genotypes_Ethiopia = chrx_genotypes.take(Ethiopia_indices, axis=1)
chr2_genotypes_Ethiopia = chr2_genotypes.take(Ethiopia_indices, axis=1)
chr3_genotypes_Ethiopia = chr3_genotypes.take(Ethiopia_indices, axis=1)

chrx_genotypes_Yemen = chrx_genotypes.take(Yemen_indices, axis=1)
chr2_genotypes_Yemen = chr2_genotypes.take(Yemen_indices, axis=1)
chr3_genotypes_Yemen = chr3_genotypes.take(Yemen_indices, axis=1)

# before the main loops, define pop for each sample name
# e.g. a dict mapping sample name → its population
pop_of = {
    'SRR15257906':'India', 'SRR15257907':'India', 'SRR15257908':'India', 'SRR15257909':'India',
       'SRR15257910':'India', 'SRR15257911':'India', 'SRR15257912':'India', 'SRR15257913':'India',
       'SRR15257914':'India', 'SRR15257915':'India', 'SRR15293885':'India', 'SRR15293886':'India',
       'SRR15293887':'India', 'SRR15293888':'India', 'SRR15293889':'India', 'SRR15293890':'India',
       'SRR15293891':'India', 'SRR15293892':'India', 'SRR15293893':'India', 'SRR15293894':'India',
       'X1296':'Ethiopia', 'X1307':'Ethiopia', 'X1402':'Ethiopia', 'X1403':'Ethiopia', 'X1404':'Ethiopia', 'X1408':'Ethiopia', 'X1409':'Ethiopia',
       'X1410':'Ethiopia', 'X1415':'Yemen', 'X1416':'Yemen', 'X1417':'Yemen', 'X1419':'Yemen', 'X1420':'Yemen', 'X1421':'Yemen',
       'X1423':'Yemen', 'X1424':'Yemen', 'X1425':'Yemen', 'X1580':'Ethiopia', 'X1581':'Ethiopia', 'X1583':'Ethiopia', 'X1585':'Yemen',
       'X1586':'Yemen', 'X1587':'Yemen', 'X1604':'Ethiopia', 'X1605':'Ethiopia', 'X1673':'Ethiopia', 'X1676':'Ethiopia', 'X1679':'Ethiopia',
       'X1680':'Ethiopia', 'X1735':'Ethiopia', 'X1736':'Ethiopia', 'X1738':'Ethiopia', 'X1740':'Ethiopia', 'X1742':'Ethiopia', 'X1743':'Ethiopia',
       'X1747':'Ethiopia'
}

samples

In [ ]:
# Parameters for roh_mhmm
phet_roh      = 0.001
phet_nonroh   = (0.005, 0.02)
transition    = 1e-06
min_roh       = 100_000

# Define populations and chromosomes
populations   = ["India", "Ethiopia", "Yemen"]
chromosomes   = ["chrx", "chr2", "chr3"]

# This dictionary will hold: results_roh[pop][chrom] = DataFrame of all runs for that pop/chrom
results_roh = {}
for pop in populations:
    results_roh[pop] = {}

for chrom in chromosomes:
    # open or reference your callset for this chrom
    callset = locals()[f"{chrom}_callset"]  

    # get that chromosome’s sample names
    samples = callset['samples'][:]               # shape = (n_samples,)
    # build a boolean mask for which belong to each pop
    for pop in populations:
        mask = np.vectorize(lambda s: pop_of[s])(samples) == pop
        sample_idxs = np.nonzero(mask)[0]         # these are now valid 0..n_samples-1

        # grab the GenotypeArray & positions
        genotypes = locals()[f"{chrom}_genotypes"]  
        positions = locals()[f"{chrom}_positions"]
        contig_size = locals()[f"{chrom}_size"]

        dfs = []
        for si in sample_idxs:
            gv = genotypes[:, si, :]
            roh_df, froh = allel.roh_mhmm(
                gv, positions,
                # phet_roh=phet_roh,
                # phet_nonroh=phet_nonroh,
                # transition=transition,
                min_roh=min_roh,
                is_accessible=None,
                contig_size=contig_size
            )
            roh_df['sample'] = samples[si]  # store the sample name
            roh_df['froh']   = froh
            dfs.append(roh_df)

        # concatenate and store
        all_roh = pd.concat(dfs, ignore_index=True)
        results_roh[pop][chrom] = all_roh
        
        
# Now results_roh["India"]["chr2"] is a DataFrame containing all RoH runs
# for every Indian sample on chromosome 2, with columns:
#   ['start','stop','length','is_marginal','sample','froh']


In [ ]:
# Assuming `results_roh` dictionary is already in the environment as defined previously.

for pop, chrom_dict in results_roh.items():
    for chrom, df in chrom_dict.items():
        # Create figure with two subplots: ROH length distribution and fROH per sample
        fig, axes = plt.subplots(2, 1, figsize=(12, 8))

        # Histogram of ROH lengths
        sns.histplot(df['length'], bins=50, ax=axes[0])
        axes[0].set_title(f'{pop} - {chrom}: ROH Length Distribution')
        axes[0].set_xlabel('ROH length (bp)')
        axes[0].set_ylabel('Count')

        # Barplot of fROH per sample
        froh_df = df[['sample', 'froh']].drop_duplicates().sort_values('froh')
        sns.barplot(x='sample', y='froh', data=froh_df, ax=axes[1])
        axes[1].set_title(f'{pop} - {chrom}: Fraction of Genome in ROH (fROH)')
        axes[1].set_xlabel('Sample')
        axes[1].set_ylabel('fROH')
        axes[1].tick_params(axis='x', rotation=45)

        plt.tight_layout()
        plt.show()


In [ ]:
for pop in populations:
    for chrom in chromosomes:
        results_roh[pop][chrom]['length'].plot(kind='kde') # , bins=5, edgecolor='black'
        plt.title('Distribution of RoH lengths (Histogram)')
        plt.xlabel('Length')
        plt.ylabel('Frequency')
        plt.show()

In [ ]:
# your input: results_roh[pop][chrom] → DataFrame with columns
# ['start','stop','length','is_marginal','sample','froh']

window_size = 1000  # 1 kb windows

# output dict
probabilities = {}

for pop, chrom_dict in results_roh.items():
    probabilities[pop] = {}
    
    for chrom, roh_df in chrom_dict.items():
        # 1) find contig length (use max 'stop' position as a lower bound)
        contig_len = int(roh_df['stop'].max())
        
        # 2) tile the chromosome into non-overlapping windows:
        #    [0–1000), [1000–2000), …, up to contig_len
        window_starts = np.arange(0, contig_len, window_size)
        
        # 3) get the set of all samples in this pop × chrom
        all_samples = roh_df['sample'].unique()
        n_samples = len(all_samples)
        
        rows = []
        for ws in window_starts:
            we = ws + window_size
            # a) select RoH segments overlapping [ws, we)
            ov = roh_df[
                (roh_df['stop']  >= ws) &
                (roh_df['start'] <  we)
            ]
            # b) count how many distinct samples have ≥1 overlapping RoH
            n_overlap = ov['sample'].nunique()
            prop = n_overlap / n_samples
            
            rows.append({
                'window_start': ws,
                'window_end':   we,
                'proportion':   prop
            })
        
        # 4) assemble into DataFrame and store
        prob_df = pd.DataFrame(rows)
        probabilities[pop][chrom] = prob_df
        
        # 5) quick line‐plot of proportion vs position
        plt.figure(figsize=(8, 2.5))
        plt.plot(
            prob_df['window_start'],
            prob_df['proportion'],
            lw=1
        )
        plt.ylim(0, 1)
        plt.xlabel('Genomic position (bp)')
        plt.ylabel('Fraction in RoH')
        plt.title(f'{pop}: {chrom}')
        plt.tight_layout()
        plt.show()
